In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

import os
import glob

from scipy.ndimage import gaussian_filter1d

from scipy import signal

from skimage import io, color

In [2]:
imgDir = "images/"
saveDir = "output/"
maskDir = "masks/"

In [9]:
def get_patch(center_pixel, img, patch_size):
    """
    Get a patch of size (patch_size x patch_size) centered around the center_pixel.

    Args:
        center_pixel (tuple): (x, y) coordinates of the center of the patch.
        img (numpy.ndarray): The input image as a NumPy array.
        patch_size (int): The size of the square patch to extract.

    Returns:
        numpy.ndarray: A patch of size (patch_size x patch_size) centered around center_pixel.
        The patch may be smaller near the image boundaries.
    """
    # Half size for determining the range
    half_size = patch_size // 2

    # Extract center coordinates
    center_x, center_y = center_pixel

    # Determine the row and column bounds for the patch
    start_x = max(center_x - half_size, 0)
    end_x = min(center_x + half_size + 1, img.shape[0])

    start_y = max(center_y - half_size, 0)
    end_y = min(center_y + half_size + 1, img.shape[1])

    # Return the extracted patch

    return img[start_x:end_x, start_y:end_y]

In [10]:
def compute_isophote(patch):
    """
    Compute the isophote direction for a given patch of an image.

    Args:
        patch (numpy.ndarray): A 2D array representing the intensity values in the patch.

    Returns:
        numpy.ndarray: A 2D vector representing the isophote direction.
    """
    if patch.ndim != 2:
        raise ValueError("Patch must be a 2D array of intensity values.")

    # Compute gradients in x and y directions
    grad_y, grad_x = np.gradient(patch)
    print(grad_x, grad_y)

    # Compute the gradient magnitude
    grad_magnitude = grad_x**2 + grad_y**2
    print(grad_magnitude)

    # Find the pixel with the maximum gradient magnitude
    max_index = np.unravel_index(
        np.argmax(grad_magnitude, axis=None), grad_magnitude.shape
    )

    # Gradient vector at the point of maximum gradient magnitude
    max_grad = np.array([grad_x[max_index], grad_y[max_index], 0])

    # Compute the isophote direction as the cross product with the z-axis
    isophote = np.cross(max_grad, np.array([0, 0, 1]))

    # Normalize the isophote vector
    isophote_2d = isophote[:2] / np.linalg.norm(isophote[:2])

    return isophote_2d

In [11]:
def find_contours(image):
    """
    Find contours using OpenCV.

    Args:
        image (np.ndarray): Binary image.

    Returns:
        list: List of contours found in the image, where contour shape is (NumPoints, 1, 2)
    """
    contours, _ = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    contours = contours[1:]
    return contours

In [12]:
def compute_normals(contours):
    """
    Compute the normals for a set of contours.

    Parameters:
        contours (list of numpy.ndarray): Contours as returned by cv2.findContours.

    Returns:
        list of numpy.ndarray: Normals for each contour.
    """
    normals = []

    for contour in contours:
        # Flatten the contour to remove the unnecessary dimension
        points = contour.squeeze(axis=1)  # Shape: [numPoints, 2]
        num_points = len(points)
        print(contour)

        # Placeholder for normals of the current contour
        contour_normals = np.zeros((num_points, 2), dtype=np.float32)

        for i in range(num_points):
            # Get the current point and the previous point
            current_point = points[i]
            previous_point = points[i - 1]  # Wraps around to the last point

            # Compute the edge vector
            edge_vector = current_point - previous_point

            # Add the 3rd dimension (z=0) for cross product computation
            edge_vector_3d = np.array(
                [edge_vector[0], edge_vector[1], 0], dtype=np.float32
            )
            normal_3d = np.cross(edge_vector_3d, np.array([0, 0, 1], dtype=np.float32))

            # Normalize the 2D normal vector
            normal_2d = normal_3d[:2]  # Extract (x, y) from the 3D normal
            norm = np.linalg.norm(normal_2d)
            if norm != 0:
                normal_2d /= norm

            # Store the normalized normal
            contour_normals[i] = normal_2d

        # Append the normals of the current contour to the list
        normals.append(contour_normals)

    return normals

In [13]:
def compute_data(contours, data, img, patch_size):
    """
    Compute the data term for the contours using isophote alignment.

    Args:
        contours (list): Contours found in the image.
        normals (list): Normals corresponding to the contours.
        img (np.ndarray): Grayscale input image.
        patch_size (int): Size of the patch for gradient computation.

    Returns:
        np.ndarray: Data term values stored in an array of the same shape as `img`.
    """
    normals = compute_normals(contours)

    # Loop through each contour and its corresponding normals
    for contour, normal_set in zip(contours, normals):
        # Loop through each point in the contour
        for i, point in enumerate(contour):
            # Extract the pixel coordinates (row, column)
            p = tuple(reversed(point[0]))  # Convert (x, y) to (row, column)

            # Get the patch centered around the current point
            patch = get_patch(patch_size=patch_size, center_pixel=p, img=img)

            # Skip if the patch size is smaller than expected (boundary points)
            if patch.shape[0] < patch_size or patch.shape[1] < patch_size:
                continue

            # Compute the isophote direction using the patch
            isophote = compute_isophote(patch)

            # Get the normal vector at this point
            normal = normal_set[i]

            # Compute the alignment between isophote and normal
            alignment = np.abs(np.dot(isophote, normal))

            # Update the data term at the pixel
            data[p] = alignment

    return data

In [14]:
def compute_confidence(contours, confidence, mask, patch_size):
    patch_area = patch_size**2
    for contour in contours:
        for i in range(len(contour)):
            p = contour[i, 0]
            p = (p[1], p[0])
            neighbors = get_patch(img=mask, center_pixel=p, patch_size=patch_size) / 255
            confidence[p[0], p[1]] = np.sum(neighbors) / patch_area
    return confidence

In [15]:
def compute_texel_size(img):
    """
    Compute the texel size based on image size
    (make sure it's odd so that the kernel has a well-defined center).

    Args:
        img (np.ndarray): The input image.

    Returns:
        int: The size of the texel
    """
    texel_size = min(img.shape[:-1]) // 30

    # Ensure texel size is odd
    if texel_size % 2 == 0:
        texel_size += 1
    return texel_size

In [16]:
def find_best_point(contours, priority_map):
    """
    Finds the best point based on the highest priority value from the priority map.

    Parameters:
    - contours: A list of contours where each contour is a numpy array of points.
    - priority_map: A 2D numpy array containing priority values at each location.

    Returns:
    - bestPoint: A tuple (x, y) representing the best point based on the highest priority value.
    """
    highest_priority = float("-inf")
    bestPoint = (-1, -1)

    for contour in contours:
        for point in contour:
            # Convert point from (x, y) in column-first to (x, y) in row-first format
            x, y = point[0, 1], point[0, 0]
            current_priority = priority_map[x, y]

            # Check if the current point has a higher priority
            if current_priority > highest_priority:
                highest_priority = current_priority
                bestPoint = (x, y)

    return bestPoint

In [17]:
def find_best_match(p, img_lab, maskEdgeCase, size):
    """
    Find the best exemplar patch in the image based on a given point and similarity measure.

    Args:
        p (tuple): Coordinates of the target point (row, column).
        img_lab (np.ndarray): Image in LAB color space.
        maskEdgeCase (np.ndarray): Binary mask to handle edge cases where the mask is not fully within the image.
        size (int): Size of the patch to compare.

    Returns:
        bestExemplar (np.ndarray): The best matching patch in LAB color space.
        bestInd (np.ndarray): Coordinates of the top-left corner of the best exemplar patch.
    """
    # Calculate the radius of the patch
    r = size // 2

    # Create a neighbor mask by scaling the edge case mask
    neighborMask = get_patch(
        img=(np.repeat(maskEdgeCase[:, :, np.newaxis], 3, axis=2) / 255),
        center_pixel=p,
        patch_size=size,
    )

    # Get the neighborhood around the point p in the image
    neighbors = get_patch(img=img_lab, center_pixel=p, patch_size=size) * neighborMask

    bestD = np.inf
    bestExemplar = -1
    bestInd = -1

    stride = max(
        size // 9, 1
    )  # Speed up processing for large images by reducing the step size

    # Loop through the image with stride to find the best exemplar patch
    for i in range(r + 1, img_lab.shape[0] - r - 1, stride):
        for j in range(r + 1, img_lab.shape[1] - r - 1, stride):
            # Skip regions that include the target
            if (
                np.count_nonzero(
                    maskEdgeCase[i - r : i + r + 1, j - r : j + r + 1] == 0
                )
                > 0
            ):
                continue

            # Extract the patch around the current position
            neighbors_i = img_lab[i - r : i + r + 1, j - r : j + r + 1]

            # Skip if the size of the patch does not match the original neighbors shape
            if not neighbors.shape == neighbors_i.shape:
                continue

            # Calculate the distance (difference) between the target neighbors and the current neighbors
            d = np.linalg.norm((neighbors - neighbors_i * neighborMask).flatten())

            # Update the best exemplar patch if this one has a smaller distance
            if d < bestD:
                bestD = d
                bestExemplar = neighbors_i
                bestInd = np.array([i, j])

    return bestExemplar, bestInd

In [18]:
def inpaint_patch(img_lab, center_point, patch_size, exemplar):
    """
    Inpaints a patch of the image with the exemplar.

    Parameters:
    img_lab (np.ndarray): The image in LAB color space.
    p (tuple): The coordinates (row, col) of the patch center.
    patch_size.
    exemplar (np.ndarray): The exemplar patch to copy over.

    Returns:
    np.ndarray: The updated image.
    """
    r = patch_size // 2
    # Calculate the patch coordinates
    row_start = center_point[0] - r
    row_end = center_point[0] + r + 1
    col_start = center_point[1] - r
    col_end = center_point[1] + r + 1

    # Inpaint the patch in the image
    img_lab[row_start:row_end, col_start:col_end] = exemplar

    return img_lab

In [19]:
def update_mask(mask, center_point, patch_size):
    r = patch_size // 2
    mask[
        center_point[0] - r : center_point[0] + r + 1,
        center_point[1] - r : center_point[1] + r + 1,
    ] = 255
    return mask

In [26]:
if not os.path.exists(saveDir):
    os.makedirs(saveDir)

for filename in os.listdir(imgDir):
    if not glob.glob(os.path.join(maskDir, filename.split(".")[0] + "_mask.*")):
        print("Image " + filename + " has no mask. Skipping...")
        continue

    print("Reading " + filename)
    img = cv2.imread(os.path.join(imgDir, filename))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_lab = color.rgb2lab(img_rgb)
    mask = cv2.imread(os.path.join(maskDir, filename.split(".")[0] + "_mask.jpg"))
    maskGray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(maskGray, 50, 1, 0)

    patch_size = compute_texel_size(img=img)
    print("Texel size:", patch_size)

    r = patch_size // 2
    data = np.zeros(mask.shape)
    iteration = 0

    while True:
        confidence = mask
        unique_values, counts = np.unique(confidence, return_counts=True)

        # Print the results
        for value, count in zip(unique_values, counts):
            print(f"Value: {value}, Count: {count}")

        print(f"Filling region {iteration}")
        contours = find_contours(mask)

        if iteration == 20:
            break

        confidence = compute_confidence(
            contours=contours,
            confidence=confidence,
            mask=mask,
            patch_size=patch_size,
        )
        priority = confidence

        center_point = find_best_point(contours, priority)
        exemplar, exemplar_p = find_best_match(center_point, img_lab, mask, patch_size)

        img_lab[
            center_point[0] - r : center_point[0] + r + 1,
            center_point[1] - r : center_point[1] + r + 1,
        ] = exemplar
        img = (color.lab2rgb(img_lab) * 255).astype(np.uint8)
        img_lab = color.rgb2lab(img)

        mask = update_mask(mask=mask, center_point=center_point, patch_size=patch_size)
        iteration += 1

    im_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    print("Saving image:", filename)
    success = cv2.imwrite(
        os.path.join(saveDir, filename.split(".")[0] + "_final.jpg"), im_bgr
    )
    if not success:
        print("Failed to save image:", filename)
    else:
        print("Image saved successfully:", filename)

Reading 1.png
Texel size: 9
Value: 0, Count: 1797
Value: 1, Count: 63739
Filling region 0
Value: 0, Count: 1797
Value: 1, Count: 63739
Filling region 1
Value: 0, Count: 1797
Value: 1, Count: 63739
Filling region 2
Value: 0, Count: 1797
Value: 1, Count: 63739
Filling region 3
Value: 0, Count: 1797
Value: 1, Count: 63739
Filling region 4
Value: 0, Count: 1797
Value: 1, Count: 63739
Filling region 5
Value: 0, Count: 1797
Value: 1, Count: 63739
Filling region 6
Value: 0, Count: 1797
Value: 1, Count: 63739
Filling region 7
Value: 0, Count: 1797
Value: 1, Count: 63739
Filling region 8
Value: 0, Count: 1797
Value: 1, Count: 63739
Filling region 9
Value: 0, Count: 1797
Value: 1, Count: 63739
Filling region 10
Value: 0, Count: 1797
Value: 1, Count: 63739
Filling region 11
Value: 0, Count: 1797
Value: 1, Count: 63739
Filling region 12
Value: 0, Count: 1797
Value: 1, Count: 63739
Filling region 13
Value: 0, Count: 1797
Value: 1, Count: 63739
Filling region 14
Value: 0, Count: 1797
Value: 1, Count